# Resume–Role Similarity Experiments

This notebook explores heuristic approaches to measuring
similarity between a candidate’s resume and a target role
using keyword and skill overlap.

The goal is explainability, not predictive accuracy.

In [1]:
import json
from pathlib import Path
from collections import Counter
import pandas as pd

In [2]:
TAXONOMY_PATH = Path("../data/skill_taxonomy.json")

with open(TAXONOMY_PATH, "r") as f:
    SKILL_TAXONOMY = json.load(f)

In [3]:

resume_text = """
I have experience with Python, SQL, and REST APIs.
Worked on backend services using FastAPI and Docker.
Familiar with AWS basics and database design.
"""

In [4]:
def normalize_skills(text, taxonomy):
    text = text.lower()
    found_skills = set()

    for skill, meta in taxonomy.items():
        if skill in text:
            found_skills.add(skill)
            continue
        for alias in meta.get("aliases", []):
            if alias in text:
                found_skills.add(skill)
                break

    return found_skills

resume_skills = normalize_skills(resume_text, SKILL_TAXONOMY)
resume_skills

{'aws', 'docker', 'fastapi', 'python', 'sql'}

In [5]:
ROLE_PATH = Path("../data/mock_job_descriptions/sde_backend.json")

with open(ROLE_PATH, "r") as f:
    role_data = json.load(f)

role_skills = set()
for desc in role_data["descriptions"]:
    role_skills.update(desc["skills"])

role_skills

{'aws', 'docker', 'fastapi', 'postgresql', 'python', 'rest api', 'sql'}

In [6]:
def skill_overlap(resume_skills, role_skills):
    intersection = resume_skills & role_skills
    coverage = len(intersection) / len(role_skills)
    return intersection, coverage

matched_skills, coverage_score = skill_overlap(resume_skills, role_skills)

matched_skills, coverage_score

({'aws', 'docker', 'fastapi', 'python', 'sql'}, 0.7142857142857143)

In [7]:
missing_skills = role_skills - resume_skills
missing_skills

{'postgresql', 'rest api'}

In [8]:
def ats_readiness_score(skill_coverage):
    if skill_coverage >= 0.75:
        return "Strong alignment"
    elif skill_coverage >= 0.5:
        return "Moderate alignment"
    else:
        return "Low alignment"

ats_readiness_score(coverage_score)

'Moderate alignment'

In [9]:
pd.DataFrame({
    "category": ["Matched Skills", "Missing Skills", "Coverage"],
    "details": [
        ", ".join(sorted(matched_skills)),
        ", ".join(sorted(missing_skills)),
        f"{coverage_score:.2%}"
    ]
})

,category,details
0,Matched Skills,"aws, docker, fastapi, python, sql"
1,Missing Skills,"postgresql, rest api"
2,Coverage,71.43%


In [10]:
def role_proximity(coverage):
    if coverage >= 0.75:
        return "Strong Fit"
    elif coverage >= 0.5:
        return "Near Fit"
    else:
        return "Long-Term Target"

role_proximity(coverage_score)

'Near Fit'

### Observations

- Skill overlap provides a clear and interpretable similarity signal.
- Missing skills can be directly mapped to learning recommendations.
- Heuristic thresholds are easy to explain and adjust.
- This approach avoids proprietary ATS assumptions.

### Why this matters

This notebook validates the core logic behind:
- resume alignment estimation,
- role proximity categorization,
- explainable ATS readiness indicators.

The same logic can be reused directly in backend services.